In [ ]:
#!/usr/bin/env python
# coding: utf-8


import PySimpleGUI as sg
import pandas as pd
import datetime
import os
import re

import pyodbc
import pandas.io.sql as psql
import win32com.client
from pathlib import Path



def create_report():

    # from datetime import datetime
    print('старт')
    
    data_start_form2 = data_start.replace('.', '-')
    
    text_elem = window['-text-']
    text_elem.update("Отчет формируется, не нажимать")
    
    
    sql = f'''
    SET NOCOUNT ON;
    SET DATEFORMAT dmy;
    drop table if exists #ak_work_table
--------------------------------------------------------------------------------------------------
	--1 запрос  Михаила
    SELECT * INTO #ak_work_table
    FROM
    (
    select  --COUNT(*)
            RQ.id						AS RQ_ID
            ,SUBSTRING(rg.NKOD, 1, 25)	AS reglament
            ,s.id				AS status_
            ,rq.Date_Request    AS dt
            ,rq.Date_Export     AS Date_Export
			,rq.wait_param		AS wait_param

    from SMEV_REQUEST rq (nolock)
    join f6_smev_request f6s  (nolock)on f6s.smev_request_id=rq.id
    join f6  (nolock)on f6.id=f6s.f6_id and DATRZ>='{data_start}' and (KOD=1022700000000 )
    join SMEV_REGLAM rg  (nolock)on rq.SMEV_REGLAM_ID=rg.ID 
    left join SMEV_QUERIES q  (nolock)on q.SMEV_REQUEST_ID=rq.id
    left join SMEV_STATUS s  (nolock) on s.id=q.state

    UNION
    --2 запрос  Михаила
    SELECT   
            --COUNT(*)
            SRQ.ID
            , SUBSTRING(SR.NKOD, 1, 25) AS reglament
            , s.id
            , SRQ.Date_Request
            ,SRQ.Date_Export
			,SRQ.wait_param

    FROM SMEV_REQUEST (nolock) SRQ
    join SMEV_REGLAM (nolock) SR 
    on SRQ.SMEV_REGLAM_ID = sr.id 
        --AND NKOD  LIKE  'ЗАГС%'
        --and sr.ID In (1431,1432,1433,1434,1390) 
        and User_Request in (22,327)
    left join SMEV_QUERIES (nolock) q on q.SMEV_REQUEST_ID=SRQ.id
    left join SMEV_STATUS (nolock) s  on s.id=q.state
    WHERE SRQ.Date_Request>'{data_start}'

    UNION
   --3 запрос  Михаила 
    select  
            RQ.id				       AS RQ_ID
            ,SUBSTRING(rg.NKOD, 1, 25) AS reglament
            ,s.id				AS status_
            ,rq.Date_Request    AS dt
            ,rq.Date_Export     
			,RQ.wait_param

    from SMEV_REQUEST rq (nolock)
    join SMEV_REGLAM rg  (nolock) on rq.SMEV_REGLAM_ID=rg.ID --and NKOD  LIKE  '3834R%'
    left join SMEV_QUERIES q  (nolock) on q.SMEV_REQUEST_ID=rq.id
    left join SMEV_STATUS s  (nolock) on s.id=q.state

    where 1=1 
		and Date_Request >'{data_start}' 
		and f2_id in 
					(select p.f2_id from f_pensfam p
								join f6  (nolock) on f6.id=p.decl_id and f6.DATRZ>='{data_start}' and f6.KOD=1022700000000)
    ) T1;
--------------------------------------------------------------------------------------------------

;
drop table if exists #temp_table_load


--------------------------------------------------------------------------------------------------
SELECT DISTINCT T1.RQ_ID,
		iif(status_ in (1, 4, 9), 1, 0)  as 'Ожидание запроса службы СМЭВ №1-3',
		iif(status_ in (101, 102), 1, 0) as 'Завершение опроса служб СМЭВ',
		iif(status_ in (2, 5, 10), 1, 0) as 'Получен ответ с сообщением об ошибке от службы СМЭВ №2',
		iif(status_ in (6, 7, 11), 1, 0) as 'Получен ответ от службы СМЭВ №1-3 (данные не найдены)',
		iif(status_ in (12, 13), 1, 0)   as 'Отменен запрос к службе СМЭВ №1-2',
                    case
						when substring(T1.reglament, 1, 3) != 'МВД'
						then substring(T1.reglament, 1, 4)
						else substring(T1.reglament, 1, 25)
					end as reglament,
					--substring(T1.reglament, 1, 26) as reglament, 
                    T1.status_,
                    dt, 
                    iif (wait_param is not null, 1, 0) AS smev_wait, 
					case
						when dateadd(day, 7, Date_Export) < getdate() and status_ in (1, 4, 9)
						then 1
						else 0
						end as chack_late_1_0,
					case 
						when (t1.Date_Export is null) and (wait_param is null) --[smev_wait_param] есть столбец в SMEV_REQUEST_ID
						then 1
						else 0
						end as [Date_Export], --waiting_export_of_ASP
					case 
						when (RQ_ID in     (select SMEV_REQUEST_ID from SMEV_QUERIES )) and (wait_param is null) -- not in (1, 4, 9) не нужно учитыватьт т.к. ниже их сумму вычитаем.
						then 1
						else 0
						end as [answ], --требует перепроверки, проверил
					case
						when (RQ_ID not in (select SMEV_REQUEST_ID from SMEV_QUERIES)) and (wait_param is null)
						then 1
						else 0 
						end as [not_answ] --требует перепроверки 
					into #temp_table_load
    from #ak_work_table T1
    WHERE 1=1 and
					CONCAT(T1.RQ_ID, dt) in (SELECT CONCAT(#ak_work_table.RQ_ID, max(dt)) FROM #ak_work_table group by RQ_ID)
                and t1.reglament != 'Занятость_3928R Дата приз'--'Занятость_3928R Дата признания безработным (Свердловская обл.) v1.20Prod'
				and t1.reglament != 'МВД_Cведения о авто- мото'--'МВД_Cведения о авто- мото- трансп. средствах'
				and not exists (select 1 from #ak_work_table t11 where t1.RQ_ID = t11.RQ_ID and t1.dt < '01.06.2021' and substring(t11.reglament, 1, 4) = 'ФСИН')
--------------------------------------------------------------------------------------------------

-----------------------------------3
--------------------------------------------------------------------------------------------------
select case 
	when reglament = '3834'
	then 'Администрация г. Екатеринбурга (выписка из домовой книги)'
	when reglament = 'ДТиЗ'
	then 'Департамент труда и занятости'
	when reglament = 'Заня'
	then 'Департамент труда и занятости'
	when reglament = 'Роср'
	then 'Росреестр'
	when reglament = 'ФГИС'
	then 'ФГИС ФРИ'
	when reglament = 'МВД_3418F Получение регис'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'ФМС_'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_3418F Проверка регист'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_3418F_Запрос паспортн'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_Запрос сведений о наз'
	then 'МВД (Сведения о пенсии)'
	when reglament = 'МВД_Проверка действит. па'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_Сведения о наличии су'
	then 'МВД (Сведения о наличии судимости)'
	when reglament = 'МВД_Получение регистрации'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_Сведения о назначенны'
	then 'МВД (Сведения о пенсии)'
	when reglament = 'ФССП'
	then 'ФССП (Федеральная служба судебных приставов_'
	when reglament = 'ФТС_'
	then 'ФТС (Федеральная таможенная служба), МО (Министерство обороны) сведения о пенсиях'
	when reglament = 'МО_С'
	then 'ФТС (Федеральная таможенная служба), МО (Министерство обороны) сведения о пенсиях'
	when reglament = 'ГИБД'
	then 'МВД (Cведения о авто, мото транспорте)'
	when reglament = 'МВД_3418F_проверка дейст.'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'ФСИН'
	then 'ФСИН (*информация от 01.06.2021)'
	else replace(replace(reglament, '-', ''), '_', '')
	end as [Государственные органы (организации) участвующие в предоставлении государственной услуги],
	 count(distinct RQ_ID)                                           as [Количество СМЭВ запросов всего],
	 (sum(answ) - sum([Ожидание запроса службы СМЭВ №1-3]))          as [количество полученных ответов на запросы],
	 sum([Завершение опроса служб СМЭВ])                             as [Положительные, Ответ от службы получен. Данные помещены в базу данных.],
	 sum([Получен ответ от службы СМЭВ №1-3 (данные не найдены)])    as [Положительные. Служба СМЭВ вернула результат: данные не найдены],
	(sum([Получен ответ с сообщением об ошибке от службы СМЭВ №2])+sum([Отменен запрос к службе СМЭВ №1-2])) as [Получен ответ с сообщением об ошибке от службы СМЭВ],
	sum([Ожидание запроса службы СМЭВ №1-3])                         as [Находятся на рассмотрении, всего],
	sum([Date_Export])                                               as [Ожидающие выгрузку из АСП],
	sum(chack_late_1_0)                                              as [Находятся на рассмотрении, нарушение срока],
	sum([not_answ]) - sum([Date_Export])                             as [Ответы не загружены из точки интеграции],
	sum(smev_wait)                                                   as [Ожидают заполнения обязательных параметров в АСП]

from #temp_table_load

group by case 
	when reglament = '3834'
	then 'Администрация г. Екатеринбурга (выписка из домовой книги)'
	when reglament = 'ДТиЗ'
	then 'Департамент труда и занятости'
	when reglament = 'Заня'
	then 'Департамент труда и занятости'
	when reglament = 'Роср'
	then 'Росреестр'
	when reglament = 'ФГИС'
	then 'ФГИС ФРИ'
	when reglament = 'МВД_3418F Получение регис'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'ФМС_'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_3418F Проверка регист'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_3418F_Запрос паспортн'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_Запрос сведений о наз'
	then 'МВД (Сведения о пенсии)'
	when reglament = 'МВД_Проверка действит. па'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_Сведения о наличии су'
	then 'МВД (Сведения о наличии судимости)'
	when reglament = 'МВД_Получение регистрации'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'МВД_Сведения о назначенны'
	then 'МВД (Сведения о пенсии)'
	when reglament = 'ФССП'
	then 'ФССП (Федеральная служба судебных приставов_'
	when reglament = 'ФТС_'
	then 'ФТС (Федеральная таможенная служба), МО (Министерство обороны) сведения о пенсиях'
	when reglament = 'МО_С'
	then 'ФТС (Федеральная таможенная служба), МО (Министерство обороны) сведения о пенсиях'
	when reglament = 'ГИБД'
	then 'МВД (Cведения о авто, мото транспорте)'
	when reglament = 'МВД_3418F_проверка дейст.'
	then 'МВД, ФМС (регистрация граждан)'
	when reglament = 'ФСИН'
	then 'ФСИН (*информация от 01.06.2021)'
	else replace(replace(reglament, '-', ''), '_', '')
	end 
order by [Государственные органы (организации) участвующие в предоставлении государственной услуги]
--------------------------------------------------------------------------------------------------
drop table #temp_table_load
drop table #ak_work_table


    '''
    sql = str(sql)
    import pyodbc

#     cnxn = pyodbc.connect('DRIVER={SQL Server}; SERVER=oirc-srv-sql15; DATABASE=SVO_ETL; Trusted_Connection=True')
    cnxn = pyodbc.connect('DRIVER={SQL Server}; SERVER=oirc-srv-test03; DATABASE=EVD_SVO_ETL; Trusted_Connection=True')
    cursor = cnxn.cursor()

#     %%time

    
    
    df_0 = pd.read_sql(sql, cnxn)
    cnxn.close()

#     df_0.columns = ['RQ_ID', 'reglament', 'status_', 'dt', 'answ', 'smev_wait', 'Date_Export']

#     #Ищим опоздавшие СМЭВ запросы

#     # добавляем столбец для обозначающий время за которое должен быть быть обработан запрос
#     df_0['dt'] = pd.to_datetime(df_0['dt'])
#     df_0['dt_end'] = df_0.dt + pd.Timedelta(days=7)

#     # берём дату сегодня
#     to_day = datetime.datetime.today()

#     df_0['chack_late'] = (df_0['dt_end'] < to_day)
    
#     df_0['chack_late_1_0']  = df_0['chack_late'].astype(int) & (df_0['status_'].apply(lambda x: x in [1, 4, 9])).astype(int)

#     # переназначаю поле 
#     df_0['Date_Export'] = ((df_0['smev_wait'] == 0) & (df_0['Date_Export'] == 0))

#     # переназначаем поле 
#     df_0['answ']        = ((df_0['smev_wait'] == 0) & (df_0['answ'] == 0))


#     # Убираю этот регламент записи 
#     # обновление 30.06.2021
# #     df_0 = df_0.loc[df_0['reglament'] != 'Занятость_3928R Дата признания безработным (Свердловская обл.) v1.20Prod', :]
    
#     mvd_list = []
#     def make_cut_reglam(name):
#         global mvd_df
#         name = str(name)

#         if  name[:3] != "МВД":
#             name = name[:4]
#         else:
#             mvd_list.append(name)

#             name = name[:25]
#         return  name

#     df_0['название'] = df_0.reglament.apply(make_cut_reglam)

#     # Добавляю условие ФСИН считаю с 01.06.2021

#     df_0 = df_0.loc[~((df_0['название'] == "ФСИН")&(df_0['dt'] < datetime.datetime(2021, 6, 1)))]

#     # Пивоты

#     df_piv = pd.pivot_table(data=df_0, index='название', columns='status_', values='RQ_ID', aggfunc='count').reset_index()

#     #Делаю второй пивот

#     df_piv_answ_2 = pd.pivot_table(data=df_0, index='название', columns='answ', values='RQ_ID', aggfunc='count').reset_index()                      .rename({True:'Не_ответили', False:'Ответили'}, axis=1)

#     #Делаю третий пивот статистику по опозданиям 

#     df_piv_late_3 = pd.pivot_table(data=df_0, index='название', columns='chack_late_1_0', values='RQ_ID', aggfunc='count').reset_index()                    .rename({0:'Не_опоздал', 1:'Опоздал'}, axis=1)

#     #Делаю четвёртый пивот

#     df_piv_late_4 = pd.pivot_table(data=df_0, index='название', 
#                                    columns='smev_wait', values='RQ_ID', 
#                                    aggfunc='count')\
#                         .reset_index()\
#                         .rename({1:'smew_wait', 0:'not_in_smew_wait_param'}, axis=1)

#     #Делаю пятый пивот 

#     df_piv_late_5 = pd.pivot_table(data=df_0, index='название', 
#                                    columns='Date_Export', values='RQ_ID', 
#                                    aggfunc='count')\
#                         .reset_index()\
#                         .rename({0:'has_date_export', 1:'waiting_export_of_ASP'}, axis=1)
#     # waiting_export_of_ASP = не имеющие дату выгрузки

#     # 1 пивот
#     df = df_piv.loc[:]
#     df['название_0'] = df['название']
#     df = df_piv.groupby('название', as_index=False).sum()

#     # 2 пивот
#     df_2_2 = df_piv_answ_2.loc[:]
#     df_2_2 = df_piv_answ_2.groupby('название', as_index=False).sum()

#     # 3 пивот
#     df_2_3 = df_piv_late_3.loc[:]
#     df_2_3 = df_piv_late_3.groupby('название', as_index=False).sum()

#     # 4 пивот 
#     df_2_4 = df_piv_late_4.loc[:]
#     df_2_4 = df_piv_late_4.groupby('название', as_index=False).sum()

#     # 5 пивот
#     df_2_5 = df_piv_late_5.loc[:]
#     df_2_5 = df_piv_late_5.groupby('название', as_index=False).sum()

#     df = df.merge(df_2_2, on='название', how='left')
#     df = df.merge(df_2_3, on='название', how='left')
#     df = df.merge(df_2_4, on='название', how='left')
#     df = df.merge(df_2_5, on='название', how='left')

#     df = df.loc[df['название'] != 'МВД_Cведения о авто- мото', :]
#     # обновление 17.05.2021
#     # 'МВД_Cведения о авто- мото':'МВД (Cведения о авто, мото транспорте)' 
#     # строка удалена
#     # ГИБД заменена замена ниже) 
    

#     df.replace({'название':

#     {'3834':'Администрация г. Екатеринбурга (выписка из домовой книги)',
#     'ДТиЗ':'Департамент труда и занятости',
#     'Заня':'Департамент труда и занятости',
#     'Роср':'Росреестр',
#     'ФГИС':'ФГИС ФРИ'}}
#                            , inplace=True)


#     df.replace({'название':{'МВД_3418F Получение регис':'МВД, ФМС (регистрация граждан)',                                                   'ФМС_':'МВД, ФМС (регистрация граждан)',                             'МВД_3418F Проверка регист':'МВД, ФМС (регистрация граждан)',                             'МВД_3418F_Запрос паспортн':'МВД, ФМС (регистрация граждан)',                             'МВД_Запрос сведений о наз':'МВД (Сведения о пенсии)',                             'МВД_Проверка действит. па':'МВД, ФМС (регистрация граждан)',                             'МВД_Сведения о наличии су':'МВД (Сведения о наличии судимости)',                             'МВД_Получение регистрации':'МВД, ФМС (регистрация граждан)',                             'МВД_Сведения о назначенны':'МВД (Сведения о пенсии)',                             'ФССП':'ФССП (Федеральная служба судебных приставов_',                             'ФТС_':'ФТС (Федеральная таможенная служба), МО (Министерство обороны) сведения о пенсиях',                              'МО_С':'ФТС (Федеральная таможенная служба), МО (Министерство обороны) сведения о пенсиях',                               'ГИБД':'МВД (Cведения о авто, мото транспорте)',                               'МВД_3418F_проверка дейст.':'МВД, ФМС (регистрация граждан)',                              'ФСИН':'ФСИН (*информация от 01.06.2021)'}}
#                            , inplace=True)

#     #удалаяем лишние символы из готовых названий регламентов (возможно уже все названия регламентов переделаны с от начала до конца, тогда это действие не нужно) 

#     def drop_(name):
#         name =str(name)
#         name = name.replace('_', '')
#         name = name.replace('-', '')
#         return name

#     df['название'] = df['название'].apply(drop_)

#     df_group = df.groupby('название', as_index=False).sum()

#     df_itog = pd.DataFrame()

#     df_itog['Государственные органы (организации) участвующие в предоставлении государственной услуги'] = df_group.loc[:, 'название']

#     df_itog['Количество СМЭВ запросов всего'] = df_group['Не_ответили'] + df_group['Ответили']

#     df_itog['Количество полученных ответов на запросы'] = df_group['Ответили'] - df_group[4]

#     df_itog['Положительные, Ответ от службы получен. Данные помещены в базу данных.'] =  df_group[101]
#     #Завершение опроса служб СМЭВ

#     df_itog['Положительные. Служба СМЭВ вернула результат: данные не найдены'] = df_group[6]\
#                                                                                     +df_group[7]   
#     #Получен ответ от службы СМЭВ №1 и 2 (данные не найдены)


#     df_itog['Получен ответ с сообщением об ошибке от службы СМЭВ'] =      df_group[2] \
#                                                                         + df_group[5] \
#                                                                         + df_group[12]\
#                                                                         + df_group[13]

#     df_itog['Находятся на рассмотрении, всего']  = df_group[4]
#     df_itog['Ожидающие выгрузку из АСП'] = df_group['waiting_export_of_ASP']
#     df_itog['Находятся на рассмотрении, нарушение срока'] = df_group['Опоздал']
#     df_itog['ответы не загружены из точки интеграции '] = df_group['Не_ответили'] - df_group['waiting_export_of_ASP']
#     df_itog['Ожидают заполнения обязательных параметров в АСП'] = df_group['smew_wait'] 

    
#     df_itog_2 = df_itog.iloc[:, :]

    df_itog_2 = df_0.iloc[:, :]
    print('Информация скачена')
    to_day = datetime.datetime.today()
    to_day = to_day.strftime("%d-%m-%Y")

    file_name = f'Информация по СМЭВ запросам c {data_start_form2} по {to_day}.xlsx'    
    file_path = Path(path_save).joinpath(file_name)

    df_itog_2.to_excel(file_path, header=False, startrow=2, index=False)


    # Стандарт 16 (на дату 26.05.2021)

    # Для красоты

    writer = pd.ExcelWriter(file_path,
                            engine='xlsxwriter')

    df_itog_2.to_excel(writer, sheet_name='Sheet1', startrow=1, index=False)
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    # Вписываю заголовки
    worksheet.write_row('A2', list(df_itog_2))



    #     устанавливаю ширину и высоту
    worksheet.set_row(1, 90)
    worksheet.set_column(0, 0, 50)
    for i in range(1, len(list(df_itog_2))):
        worksheet.set_column(i, i, 18)


    merge_format = workbook.add_format({
        'bold':     True,
        'border':   6,
        'align':    'center',
        'valign':   'vcenter',
        'fg_color': '#DCDCDC',
    })
    
    
    worksheet.merge_range('A1:K1', F'Информация по запросам СМЭВ  за период с {data_start_form2} по {to_day}', merge_format)


    writer.save()

    from openpyxl.utils.cell import get_column_letter
    from openpyxl import load_workbook
    from openpyxl.styles import Alignment
    from openpyxl.styles import Border
    from openpyxl.styles import Side

    wb = load_workbook(file_path)
    ws = wb.worksheets[0]


    thin = Side(border_style='thin', color='000000')

    num_row = 0
    for row in ws.iter_rows(min_row=2, min_col=1, max_col=11):
        num_row += 1
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center', wrapText=True)
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

    max_row =    ws.max_row
    max_column = ws.max_column

    for col_num in range(2, max_column+1):
        name_col = get_column_letter(col_num)
        ws[f"{name_col}{max_row+1}"] = f'=SUM({name_col}{3}:{name_col}{max_row})'
        # Расчитываю сумму. Пример: ws["B19"] = "=SUM(B3:B18)"

    for row in ws.iter_rows(min_row=3, min_col=2, max_col=11):
        for cell in row:
             cell.number_format = '#,##0'

    wb.save(file_path)
    
    done = True
    text_elem.update(f"Отчет сформирован можете выходить  done={done}")

    if not values['-email-']:
        return 1
#     Отправка писем
#     email_addresses = ['kirill.arda@yandex.ru', 'i.kozlova@egov66.ru']
    email_addresses = email_save.replace(',', ' ').split()
    outlook = win32com.client.Dispatch("Outlook.Application")

    
    for email in email_addresses:
        try:
            mail = outlook.CreateItem(0)
            mail.To = email
            mail.Subject = f'СМЭВ {to_day}'           #Тема
            mail.Body = """Отчет по смэв запросам """ #Текст
            mail.Attachments.Add(Source=str(file_path))
            mail.Send()
            print(email, "отправлено")
        except:
            pass
    print('End')
        
try:
    with open('date_start.txt') as d_s:
        data_start = d_s.read()
except:
    data_start = '01.04.2021'
        
try:
    with open('paht_save.txt') as d_s:
        path_save = d_s.read()
except:
    path_save = ""
   
try:
    with open('email_save.txt') as d_s:
        email_save = d_s.read()
except:
    email_save = 'kirill.arda@yandex.ru, i.kozlova@egov66.ru'

        
layout = [[sg.Input(default_text=data_start, key='-new_date_start-'), sg.CalendarButton(button_text='Выбрать дату', format='%d.%m.%Y', key='-new_date_start-'), sg.Button('сохранить дату начала',enable_events=True, key='-save_date-'), sg.Text('С какой даты считаем')],
          [sg.Input(default_text=path_save, key='-new_save_path-'), sg.FolderBrowse(button_text='Выбор папки', key='-new_save_path-'), sg.Button('сохранить папку сохранения',enable_events=True, key='-save_folder-'), sg.Text('В какую папку сохранить')],
          [sg.Input(default_text=email_save, key='-new_save_email-'), sg.Button('сохранить почты',enable_events=True, key='-save_email-'), sg.Text('Ввести почтовыя адреса через запятую и сохранить')],
          [sg.Text('После сохранения следующий запуск будет с сохраненными параметрами')],
          [sg.Button('Generate',enable_events=True, key='-FUNCTION-', font='Helvetica 16'),
          sg.CB('send email', key='-email-', change_submits=True, default=True)],
          [sg.Text('Нажмите для формирования. В диспечере будет отображаться "Файл не отвечает", выполнятся может до 25 мин.', size=(60, 3), key='-text-', font='Helvetica 16')],
          [sg.Output(size=(100,10))]
         
         ]
layout2 = [
    [sg.Text('Выберети папку!', size=(35, 1))]
]

# Create the window
window = sg.Window('create report СМЭВ', layout)
window1 = sg.Window('Ошибка', layout2)
# Event loop
flag_do = True
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    date_start_new = values['-new_date_start-']
    path_save_new =  values['-new_save_path-']
    email_new =  values['-new_save_email-']
    
    
    #дата
    if date_start_new != '':
        data_start = date_start_new
    if event == '-save_date-':
        with open('date_start.txt', 'w') as f:
            f.write(data_start)
            
    #папка сохранения
    if path_save_new != '':
        path_save = path_save_new
    if event == '-save_folder-':
        with open('paht_save.txt', 'w') as f:
            f.write(path_save_new)
            
    #почтовые адреса
    if email_new != '':
        email_save = email_new
    if event == '-save_email-':
        with open('email_save.txt', 'w') as f:
            f.write(email_new)
    
    
    print(data_start)
#     file_location = 'F:/ak.ardyshev/ЗАДАЧИ/3 Отчеты по СМЭВ запросам/Отчет/'
    if os.path.isdir(path_save) is True and path_save != '':
        path_true = True
    
#     path_true = re.search(r'\S', path) #\S - любой не пробельный символ
    if event == '-FUNCTION-' and path_true is True and flag_do == True:
        flag_do = False #для отсутствия возможности перезапустить Иначе всё повиснет. 
        create_report()
    elif event == '-FUNCTION-':
        event2 = window1.read()
        if event2 in (sg.WIN_CLOSED, 'Exit'):
            break

# Close the window i.e. release resource
window.close()


#


